In [12]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.authentication import InteractiveLoginAuthentication

workspace = "quick-starts-ws-165356"
resource_name = "aml-quickstarts-165356"
subscription_id = "a24a24d5-8d87-4c8a-99b6-91ed2d2df51f"
ws = Workspace.get(subscription_id=subscription_id,
                   resource_group=resource_name,
                   name=workspace)
experiment = Experiment(workspace=ws, name="udacity-project-automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-165356
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-165356


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpuclusterautoml"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [4]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)
df

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,0,1,0,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,0,0,0,0,0,0,0,1,0,0
32947,26,0,0,0,0,5,2,135,4,999,...,0,0,0,0,0,0,0,1,0,0
32948,31,0,0,0,0,4,1,386,1,999,...,0,0,0,1,0,0,0,0,0,0


In [5]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Datastore, Dataset

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

datastore = Datastore.get(ws, 'workspaceartifactstore')
dataset = Dataset.Tabular.register_pandas_dataframe(df, datastore, "dataset_from_pandas_df", show_progress=True)

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster_name)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e85901ca-1d94-4590-a61f-026f9c7a2833/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [6]:
autoMLRun = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpuclusterautoml with default configuration
Running on remote compute: cpuclusterautoml


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_77053543-1972-41b6-90d3-c8a30aeebe48,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [9]:
best_automl_run, fitted_automl_model = autoMLRun.get_output()
print(best_automl_run)
best_automl_run_metrics = best_automl_run.get_metrics()
print(best_automl_run_metrics['accuracy'])

Package:azureml-automl-runtime, training version:1.36.1, current version:1.34.0
Package:azureml-core, training version:1.36.0.post2, current version:1.34.0
Package:azureml-dataprep, training version:2.24.4, current version:2.22.2
Package:azureml-dataprep-rslex, training version:2.0.3, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.36.0, current version:1.34.0
Package:azureml-defaults, training version:1.36.0, current version:1.34.0
Package:azureml-inference-server-http, training version:0.4.2, current version:0.3.1
Package:azureml-interpret, training version:1.36.0, current version:1.34.0
Package:azureml-mlflow, training version:1.36.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.36.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.36.0, current version:1.34.0
Package:azureml-telemetry, training version:1.36.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.36.0, current v

Run(Experiment: udacity-project-automl,
Id: AutoML_77053543-1972-41b6-90d3-c8a30aeebe48_30,
Type: azureml.scriptrun,
Status: Completed)
0.9176631259484067


In [11]:
# Register best automl model.
best_automl_run_metrics = best_automl_run.get_metrics()
print(best_automl_run_metrics)

automl_model = autoMLRun.register_model(model_name = 'automl-model',
                                  tags={'Method':'AutoML'},
                                  description='AutoML Model trained on bank marketing data')

{'average_precision_score_weighted': 0.9563970133261657, 'AUC_macro': 0.9486358172298649, 'AUC_micro': 0.98114234332149, 'matthews_correlation': 0.5545563259909967, 'precision_score_macro': 0.8026367978365215, 'average_precision_score_micro': 0.9818436429992602, 'log_loss': 0.22972494703230745, 'AUC_weighted': 0.9486358172298649, 'weighted_accuracy': 0.9582905909196302, 'f1_score_micro': 0.9176631259484067, 'norm_macro_recall': 0.5081665876687571, 'f1_score_macro': 0.775336840343831, 'precision_score_micro': 0.9176631259484067, 'balanced_accuracy': 0.7540832938343784, 'accuracy': 0.9176631259484067, 'f1_score_weighted': 0.9140923892865989, 'recall_score_weighted': 0.9176631259484067, 'average_precision_score_macro': 0.8288067191342432, 'precision_score_weighted': 0.9120658388208703, 'recall_score_macro': 0.7540832938343784, 'recall_score_micro': 0.9176631259484067, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_77053543-1972-41b6-90d3-c8a30aeebe48_30/confusion_matrix',

In [14]:
cpu_cluster.delete()